# Star Wars y Neo4j

Existe una API REST pública, [SWAPI](http://swapi.co/), con información extraida de una [Wiki de Star Wars](http://starwars.wikia.com/). Al ser una [API REST](https://en.wikipedia.org/wiki/Representational_state_transfer) tiene la información estructurada por recursos (entidades). La idea de este **ejercicio** es modelar esta API con Neo4j.

Tanto la idea como el código están basados en [este notebook](https://github.com/johnymontana/SWAPI-graph/blob/master/SWAPI.ipynb) de [William Lyon](http://www.lyonwj.com/).

## Requisitos

* Python:
  * `jupyter`
  * `requests`
  * [`py2neo`](http://py2neo.org/)
* [Neo4j 3.X](http://neo4j.com/download/other-releases/)

## El modelo de relaciones

Para simplificar un poco el trabajo veremos el modelo de relaciones que sigue esta API, y por lo tanto el que proyectaremos en Neo4j.

![Modelo de relaciones](https://i.imgur.com/8ZLrOgO.png)

## ¡A programar!

In [1]:
# Importing packages
from py2neo import authenticate, Graph, Node, NodeSelector, Relationship
import requests

# Creating constants
PLACEHOLDER = 'Placeholder'

In [2]:
# Exploring the API: What endpoints are available? Let's take a look to them
r = requests.get("http://swapi.co/api/")
r.json()

{'films': 'http://swapi.co/api/films/',
 'people': 'http://swapi.co/api/people/',
 'planets': 'http://swapi.co/api/planets/',
 'species': 'http://swapi.co/api/species/',
 'starships': 'http://swapi.co/api/starships/',
 'vehicles': 'http://swapi.co/api/vehicles/'}

In [2]:
# Connecting to Neo4j instance
authenticate('localhost', 'admin', 'admin')  # Set user and pass
graph = Graph()
selector = NodeSelector(graph)  # And creating the selector
#graph.delete_all()#borramos los grafos de Neo4j

In [8]:

# Creating uniqueness constraints based on the datamodel
graph.schema.create_uniqueness_constraint('Film', 'url')
graph.schema.create_uniqueness_constraint('Person', 'url')
graph.schema.create_uniqueness_constraint('Planets', 'url')  # We need a few more constraints
graph.schema.create_uniqueness_constraint('Species', 'url')
graph.schema.create_uniqueness_constraint('Starships', 'url')
graph.schema.create_uniqueness_constraint('Vehicles', 'url')

ConstraintViolationException: Constraint already exists: CONSTRAINT ON ( film:Film ) ASSERT film.url IS UNIQUE

In [9]:
# Some helper functions (Busca nodo inicio y final de la relación y si no existe lo crea)
def merge_one_node(*labels, **properties):
    merged = selector.select(*labels, **properties).first()
    if not merged:
        merged = Node(*labels, **properties)
        graph.create(merged)
    return merged
#Busca nodo inicio y final de la relación y si no existe lo crea
def create_unique_relationship(start_node, rel_type, end_node):
    rels = graph.match(start_node=start_node, rel_type=rel_type, end_node=end_node)
    length = 0
    for rel in rels:
        length += 1
    if length == 0:
        rel = Relationship(start_node, rel_type, end_node)
        graph.create(rel)

Hemos creado restricciones basadas en las URLs porque son únicas para cada entidad, funcionando como un `id` al uso.

Ahora vamos a ver como son las distintas recursos (entidades) de la API. Echándoles un vistazo a su estructura podremos crear funciones para mapearlas a las bases de datos.

La **idea** que vamos a seguir es la siguiente:
* Siempre que nos encontremos con una entidad la mapearemos a la base de datos con las funciones que crearemos.
* Si una entidad tiene relaciones con otras, la API solo nos devolverá la URL de estas nuevas entidades, por lo que crearemos nodos provisionales (`Placeholder`) que solo contengan su URL.
* Cuando nos encontremos con un entidad que este representada por un nodo `Placeholder`, acutalizaremos ese nodo con su información real y elminaremos su estado provisional.
* Además de las entidades de la API (`person`, `film`, `planet`, `species`, `starship` y `vehicle`) también crearemos entidades para ciertos elementos que tienen valor como tal: `director`, `producer`, `climate`, `terrain`, `manufacturer`, `starship class` y `vehicle class`.

In [10]:
# Fetch a single person entity from the API for check the attributes it has
r = requests.get("http://swapi.co/api/people/1/")
params = r.json()
params

{'birth_year': '19BBY',
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'eye_color': 'blue',
 'films': ['http://swapi.co/api/films/6/',
  'http://swapi.co/api/films/3/',
  'http://swapi.co/api/films/2/',
  'http://swapi.co/api/films/1/',
  'http://swapi.co/api/films/7/'],
 'gender': 'male',
 'hair_color': 'blond',
 'height': '172',
 'homeworld': 'http://swapi.co/api/planets/1/',
 'mass': '77',
 'name': 'Luke Skywalker',
 'skin_color': 'fair',
 'species': ['http://swapi.co/api/species/1/'],
 'starships': ['http://swapi.co/api/starships/12/',
  'http://swapi.co/api/starships/22/'],
 'url': 'http://swapi.co/api/people/1/',
 'vehicles': ['http://swapi.co/api/vehicles/14/',
  'http://swapi.co/api/vehicles/30/']}

In [11]:
# Define a function for inserting a Person entity into the graph
# For resources referenced in the Person entity (like homeworld and starships)
# we can create a relationship and a node containing only the url, so, this new 
# node acts as a placeholder that we'll need to fill in later
def create_person(params):
    person = merge_one_node('Person', url=params['url'])
    person['birth_year'] = params['birth_year']
    person['created'] = params['created']
    person['edited'] = params['edited']
    person['eye_color'] = params['eye_color']
    person['gender'] = params['gender']
    person['hari_color'] = params['hair_color']
    person['height'] = params['height']
    person['mass'] = params['mass']
    person['name'] = params['name']
    person['skin_color'] = params['skin_color']
        
    if PLACEHOLDER in person.labels():
        person.remove_label(PLACEHOLDER)
    
    graph.push(person)
    
    planet = selector.select('Planet', url=params['homeworld']).first()  # Why not use merge?
    if not planet:
        planet = Node('Planet', url=params['homeworld'])
        planet.add_label(PLACEHOLDER)
        graph.create(planet)
    create_unique_relationship(person, 'IS_FROM', planet)
    
    for species_param in params['species']:
        species = selector.select('Species', url=species_param).first()
        if not species:
            species = Node('Species', url=species_param)
            species.add_label(PLACEHOLDER)
            graph.create(species)
        create_unique_relationship(person, 'IS_SPECIES', species)
        
    ########################################
    # Here we need a block for the starships
    ########################################
    for starship_param in params['starships']:
        starship = selector.select('Starship', url=starship_param).first()
        if not starship:
            starship = Node('Starship', url=starship_param)
            starship.add_label(PLACEHOLDER)
            graph.create(starship)
        create_unique_relationship(person, 'PILOTS', starship)
    
    
    #######################################
    # Here we need a block for the vehicles
    #######################################
    for vehicle_param in params['vehicles']:
        vehicle = selector.select('Vehicle', url=vehicle_param).first()
        if not vehicle:
            vehicle = Node('Vehicle', url=vehicle_param)
            vehicle.add_label(PLACEHOLDER)
            graph.create(vehicle)
        create_unique_relationship(person, 'PILOTS', vehicle)  
    

In [12]:
# Creating Luke Skywalker
create_person(params)

In [13]:
# Fetch a single Film entity from the API
r = requests.get("http://swapi.co/api/films/1/")
params = r.json()
params

{'characters': ['http://swapi.co/api/people/1/',
  'http://swapi.co/api/people/2/',
  'http://swapi.co/api/people/3/',
  'http://swapi.co/api/people/4/',
  'http://swapi.co/api/people/5/',
  'http://swapi.co/api/people/6/',
  'http://swapi.co/api/people/7/',
  'http://swapi.co/api/people/8/',
  'http://swapi.co/api/people/9/',
  'http://swapi.co/api/people/10/',
  'http://swapi.co/api/people/12/',
  'http://swapi.co/api/people/13/',
  'http://swapi.co/api/people/14/',
  'http://swapi.co/api/people/15/',
  'http://swapi.co/api/people/16/',
  'http://swapi.co/api/people/18/',
  'http://swapi.co/api/people/19/',
  'http://swapi.co/api/people/81/'],
 'created': '2014-12-10T14:23:31.880000Z',
 'director': 'George Lucas',
 'edited': '2015-04-11T09:46:52.774897Z',
 'episode_id': 4,
 'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies man

In [14]:
# Define a function for inserting a Film entity into the graph
def create_film(params):
    film = merge_one_node('Film', url=params['url'])
    film['created'] = params['created']
    film['edited'] = params['edited']
    film['episode_id'] = int(params['episode_id'])
    film['opening_crawl'] = params['opening_crawl']
    film['release_date'] = params['release_date']
    film['title'] = params['title']
    film['name'] = params['title']  # Adding this only for the visualization
    
    if PLACEHOLDER in film.labels():
        film.remove_label(PLACEHOLDER)
    
    graph.push(film)
    
    # Important: 'director' is a string with multiple values separated by commas
    for director_params in params['director'].split(','):
        director_params = director_params.strip()
        director = merge_one_node('Director', name=director_params)
        create_unique_relationship(film, 'DIRECTED_BY', director)
        
    # Important: 'producer' is a string with multiple values separated by commas
    ########################################
    # Here we needproducer a block for the producers
    ########################################
    for producer_params in params['producer'].split(','):
        producer_params = producer_params.strip()
        producer = merge_one_node('Producer', name=producer_params)
        create_unique_relationship(film, 'PRODUCED_BY', producer)


    for character_param in params['characters']:
        character = selector.select('Person', url=character_param).first()
        if not character:
            character = Node('Person', url=character_param)
            character.add_label(PLACEHOLDER)
            graph.create(character)
        create_unique_relationship(character, 'APPEARS_IN', film)
       
    ######################################
    # Here we need a block for the planets
    ######################################
    for planet_param in params['planets']:
        planet = selector.select('Planet', url=planet_param).first()
        if not planet:
            planet = Node('Planet', url=planet_param)
            planet.add_label(PLACEHOLDER)
            graph.create(planet)
        create_unique_relationship(planet, 'TAKES_PLACE_ON', film)
    
    ######################################
    # Here we need a block for the species
    ######################################
    for species_param in params['species']:
        species = selector.select('Species', url=species_param).first()
        if not species:
            species = Node('Species', url=species_param)
            species.add_label(PLACEHOLDER)
            graph.create(species)
        create_unique_relationship(species, 'APPEARS_IN', film)   
    ########################################
    # Here we need a block for the starships
    ########################################
    for starship_param in params['starships']:
        starship = selector.select('Starship', url=starship_param).first()
        if not starship:
            starship = Node('Starship', url=starship_param)
            starship.add_label(PLACEHOLDER)
            graph.create(starship)
        create_unique_relationship(starship, 'APPEARS_IN', film)
    
    
    #######################################
    # Here we need a block for the vehicles
    #######################################
    for vehicle_param in params['vehicles']:
        vehicle = selector.select('Vehicle', url=vehicle_param).first()
        if not vehicle:
            vehicle = Node('Vehicle', url=vehicle_param)
            vehicle.add_label(PLACEHOLDER)
            graph.create(vehicle)
        create_unique_relationship(vehicle, 'APPEARS_IN', film)  

In [15]:
# Creating A New Hope
create_film(params)

In [16]:
# Fetch a single Planet entity from the API
r = requests.get("http://swapi.co/api/planets/1/")
params = r.json()
params

{'climate': 'arid',
 'created': '2014-12-09T13:50:49.641000Z',
 'diameter': '10465',
 'edited': '2014-12-21T20:48:04.175778Z',
 'films': ['http://swapi.co/api/films/5/',
  'http://swapi.co/api/films/4/',
  'http://swapi.co/api/films/6/',
  'http://swapi.co/api/films/3/',
  'http://swapi.co/api/films/1/'],
 'gravity': '1 standard',
 'name': 'Tatooine',
 'orbital_period': '304',
 'population': '200000',
 'residents': ['http://swapi.co/api/people/1/',
  'http://swapi.co/api/people/2/',
  'http://swapi.co/api/people/4/',
  'http://swapi.co/api/people/6/',
  'http://swapi.co/api/people/7/',
  'http://swapi.co/api/people/8/',
  'http://swapi.co/api/people/9/',
  'http://swapi.co/api/people/11/',
  'http://swapi.co/api/people/43/',
  'http://swapi.co/api/people/62/'],
 'rotation_period': '23',
 'surface_water': '1',
 'terrain': 'desert',
 'url': 'http://swapi.co/api/planets/1/'}

In [17]:
# Define a function for inserting a Planet entity into the graph
def create_planet(params):
    planet = merge_one_node('Planet', url=params['url'])
    planet['created'] = params['created']
    planet['diameter'] = params['diameter']
    planet['edited'] = params['edited']
    planet['gravity'] = params['gravity']
    planet['name'] = params['name']
    planet['orbital_period'] = params['orbital_period']
    planet['population'] = params['population']
    planet['rotation_period'] = params['rotation_period']
    planet['surface_water'] = params['surface_water']
    
    if PLACEHOLDER in planet.labels():
        planet.remove_label(PLACEHOLDER)
    
    graph.push(planet)
    
    # Important: 'climate' is a string with multiple values separated by commas
    for climate_params in params['climate'].split(','):
        climate_params = climate_params.strip()
        climate = merge_one_node('Climate', type=climate_params)
        create_unique_relationship(planet, 'HAS_CLIMATE', climate)
        
    # Important: 'terrain' is a string with multiple values separated by commas
    #######################################
    # Here we need a block for the terrains
    #######################################
    for terrain_params in params['terrain'].split(','):
        terrain_params = terrain_params.strip()
        terrain = merge_one_node('Terrain', type=terrain_params)
        create_unique_relationship(planet, 'HAS_TERRAIN', terrain)

In [18]:
# Creating Tatooine
create_planet(params)

In [19]:
# Fetch a single Film entity from the API
r = requests.get("http://swapi.co/api/species/2/")
params = r.json()
params

{'average_height': 'n/a',
 'average_lifespan': 'indefinite',
 'classification': 'artificial',
 'created': '2014-12-10T15:16:16.259000Z',
 'designation': 'sentient',
 'edited': '2015-04-17T06:59:43.869528Z',
 'eye_colors': 'n/a',
 'films': ['http://swapi.co/api/films/7/',
  'http://swapi.co/api/films/5/',
  'http://swapi.co/api/films/4/',
  'http://swapi.co/api/films/6/',
  'http://swapi.co/api/films/3/',
  'http://swapi.co/api/films/2/',
  'http://swapi.co/api/films/1/'],
 'hair_colors': 'n/a',
 'homeworld': None,
 'language': 'n/a',
 'name': 'Droid',
 'people': ['http://swapi.co/api/people/2/',
  'http://swapi.co/api/people/3/',
  'http://swapi.co/api/people/8/',
  'http://swapi.co/api/people/23/',
  'http://swapi.co/api/people/87/'],
 'skin_colors': 'n/a',
 'url': 'http://swapi.co/api/species/2/'}

In [20]:
# Define a function for inserting a Species entity into the graph
def create_species(params):
    species = merge_one_node('Species', url=params['url'])
    species['name'] = params['name']
    species['language'] = params['language']
    species['average_height'] = params['average_height']
    species['average_lifespan'] = params['average_lifespan']
    species['classification'] = params['classification']
    species['created'] = params['created']
    species['designation'] = params['designation']
    species['eye_colors'] = params['eye_colors']
    species['hair_colors'] = params['hair_colors']
    species['skin_colors'] = params['skin_colors']
    
    if PLACEHOLDER in species.labels():
        species.remove_label(PLACEHOLDER)
        
    graph.push(species)

In [21]:
# Creating Droid
create_species(params)

In [22]:
# Fetch a single Film entity from the API
r = requests.get("http://swapi.co/api/starships/2/")
params = r.json()
params

{'MGLT': '60',
 'cargo_capacity': '3000000',
 'consumables': '1 year',
 'cost_in_credits': '3500000',
 'created': '2014-12-10T14:20:33.369000Z',
 'crew': '165',
 'edited': '2014-12-22T17:35:45.408368Z',
 'films': ['http://swapi.co/api/films/6/',
  'http://swapi.co/api/films/3/',
  'http://swapi.co/api/films/1/'],
 'hyperdrive_rating': '2.0',
 'length': '150',
 'manufacturer': 'Corellian Engineering Corporation',
 'max_atmosphering_speed': '950',
 'model': 'CR90 corvette',
 'name': 'CR90 corvette',
 'passengers': '600',
 'pilots': [],
 'starship_class': 'corvette',
 'url': 'http://swapi.co/api/starships/2/'}

In [23]:
# Define a function for inserting a Starship entity into the graph
def create_starship(params):
    starship = merge_one_node('Starship', url=params['url'])
    starship['MGLT'] = params['MGLT']
    starship['consumables'] = params['consumables']
    starship['cost_in_credits'] = params['cost_in_credits']
    starship['created'] = params['created']
    starship['crew'] = params['crew']
    starship['edited'] = params['edited']
    starship['hyperdrive_rating'] = params['hyperdrive_rating']
    starship['length'] = params['length']
    starship['max_atmosphering_speed'] = params['max_atmosphering_speed']
    starship['model'] = params['model']
    starship['name'] = params['name']
    starship['passengers'] = params['passengers']
    
    if PLACEHOLDER in starship.labels():
        starship.remove_label(PLACEHOLDER)
    
    graph.push(starship)
    
    manufacturer = merge_one_node('Manufacturer', name=params['manufacturer'])
    create_unique_relationship(starship, 'MANUFACTURED_BY', manufacturer)
    
    ###############################################
    # Here we need a block for the starship_classes
    ###############################################
    starship_class = merge_one_node('StarshipClass', name=params['starship_class'])
    create_unique_relationship(starship, 'IS_CLASS', starship_class)

In [24]:
# Creating CR90 corvette
create_starship(params)

In [25]:
# Fetch a single Film entity from the API
r = requests.get("http://swapi.co/api/vehicles/4/")
params = r.json()
params

{'cargo_capacity': '50000',
 'consumables': '2 months',
 'cost_in_credits': '150000',
 'created': '2014-12-10T15:36:25.724000Z',
 'crew': '46',
 'edited': '2014-12-22T18:21:15.523587Z',
 'films': ['http://swapi.co/api/films/5/', 'http://swapi.co/api/films/1/'],
 'length': '36.8',
 'manufacturer': 'Corellia Mining Corporation',
 'max_atmosphering_speed': '30',
 'model': 'Digger Crawler',
 'name': 'Sand Crawler',
 'passengers': '30',
 'pilots': [],
 'url': 'http://swapi.co/api/vehicles/4/',
 'vehicle_class': 'wheeled'}

In [26]:
# Define a function for inserting a Vehicle entity into the graph
def create_vehicle(params):
    vehicle = merge_one_node('Vehicle', url=params['url'])
    vehicle['cargo_capacity'] = params['cargo_capacity']
    vehicle['consumables'] = params['consumables']
    vehicle['cost_in_credits'] = params['cost_in_credits']
    vehicle['created'] = params['created']
    vehicle['crew'] = params['crew']
    vehicle['edited'] = params['edited']
    vehicle['length'] = params['length']
    vehicle['model'] = params['model']
    vehicle['name'] = params['name']
    vehicle['passengers'] = params['passengers']
    
    if PLACEHOLDER in vehicle.labels():
        vehicle.remove_label(PLACEHOLDER)
    
    graph.push(vehicle)
    
    manufacturer = merge_one_node('Manufacturer', name=params['manufacturer'])
    create_unique_relationship(vehicle, 'MANUFACTURED_BY', manufacturer)
    
    ##############################################
    # Here we need a block for the vehicle_classes
    ##############################################
    vehicle_class = merge_one_node('VehicleClass', name=params['vehicle_class'])
    create_unique_relationship(vehicle, 'IS_CLASS', vehicle_class)

In [27]:
# Creating Sand crawler
create_vehicle(params)

Una vez creadas todas las funciones para crear los nodos y sus relaciones a través de entidades de la API, vamos a crear el grafo completo a través de las entidades `film`. Así no solo se crearán los nodos `film` si no que también el resto de nodos de las todas las entidades y su realciones, aunque serán provisionales.

In [121]:
# Fetch Movie entities and insert into graph 
for i in range(1, 8):
    url = 'http://swapi.co/api/films/' + str(i) + '/'
    r = requests.get(url)
    params = r.json()
    create_film(params)
    print('Inserted film: ' + str(url))

Inserted film: http://swapi.co/api/films/1/
Inserted film: http://swapi.co/api/films/2/
Inserted film: http://swapi.co/api/films/3/
Inserted film: http://swapi.co/api/films/4/
Inserted film: http://swapi.co/api/films/5/
Inserted film: http://swapi.co/api/films/6/
Inserted film: http://swapi.co/api/films/7/


In [49]:
url = 'http://swapi.co/api/films/7/'
r = requests.get(url)
params = r.json()
create_film(params)
print('Inserted film: ' + str(url))

Inserted film: http://swapi.co/api/films/7/


In [50]:
# How many Placeholder nodes are in the graph now?
placeholder_count_query = '''
MATCH (p:Placeholder)
WITH collect(DISTINCT labels(p)) AS label_list
UNWIND label_list AS labels
UNWIND labels AS label
WITH collect(DISTINCT label) AS labels
WITH FILTER(label IN labels WHERE label <> "Placeholder") AS labels
UNWIND labels AS label
MATCH (p:Placeholder) WHERE label IN labels(p)
RETURN label, count(*) AS num
'''

graph.run(placeholder_count_query).data()

[{'label': 'Person', 'num': 5},
 {'label': 'Planet', 'num': 1},
 {'label': 'Starship', 'num': 1}]

Y ahora vamos a completar el grafo buscando uno a uno los nodos provisionales y rellenándolos.

In [30]:
# Get the creation function for each label
def get_function_for_label(label):
    if (label == 'Person'):
        return create_person
    elif (label == 'Planet'):
        return create_planet
    elif (label == 'Species'):
        return create_species
    elif (label == 'Starship'):
        return create_starship
    elif (label == 'Vehicle'):
        return create_vehicle
    else:
        raise ValueError('Unknown label for entity: ' + str(label))

In [51]:
# Fetch a single Placeholder entity from the graph
# Get JSON for Placeholder entity from SWAPI
# Update entity in graph (removing Placeholder label)
# Loop until graph contains no more Placeholder nodes
placeholder_node = selector.select(PLACEHOLDER).first()
placeholder_node

while placeholder_node:
    labels = placeholder_node.labels()
    labels = [label for label in labels]
    labels.remove(PLACEHOLDER)
    label = labels.pop()
    
    url = placeholder_node['url']
    r = requests.get(url)
    params = r.json()
    create_function = get_function_for_label(label)
    create_function(params)
    placeholder_node = selector.select(PLACEHOLDER).first()


Una vez completo nuestro grafo ¡vamos a averiguar cosas sobre él!

In [77]:
# What planets are most similar to Naboo?
planet_sim_query = '''
MATCH (p:Planet {name: 'Naboo'})-[:HAS_CLIMATE]->(c:Climate)<-[:HAS_CLIMATE]-(o:Planet)
MATCH (p)-[:HAS_TERRAIN]->(t:Terrain)<-[:HAS_TERRAIN]-(o)
WITH DISTINCT o, collect(DISTINCT c.type) AS climates, collect(DISTINCT t.type) AS terrains
RETURN o.name AS planet, climates, terrains, size(climates) + size(terrains) AS sim ORDER BY sim DESC LIMIT 5
'''
graph.run(planet_sim_query).data()

[{'climates': ['temperate'],
  'planet': 'Malastare',
  'sim': 3,
  'terrains': ['mountains', 'swamps']},
 {'climates': ['temperate'],
  'planet': 'Cato Neimoidia',
  'sim': 3,
  'terrains': ['mountains', 'forests']},
 {'climates': ['temperate'],
  'planet': 'Muunilinst',
  'sim': 3,
  'terrains': ['mountains', 'forests']},
 {'climates': ['temperate'],
  'planet': 'Endor',
  'sim': 3,
  'terrains': ['mountains', 'forests']},
 {'climates': ['temperate'],
  'planet': 'Alderaan',
  'sim': 2,
  'terrains': ['mountains']}]

In [16]:
# Who drove the same ships that Luke? Let's get also their manufacturers and classes.
vehicle_luke_query = '''
MATCH (l:Person {name: 'Luke Skywalker'})-[:PILOTS]->(v:Vehicle)
MATCH (p:Person)-[:PILOTS]->(v)
MATCH (v)-[:MANUFACTURED_BY]->(m:Manufacturer)
MATCH (v)-[:IS_CLASS]->(c:VehicleClass)
WHERE p.name <> l.name
RETURN  v.name as vehicle,p.name as pilot, m.name AS manufacturer ,c.name as class
'''
graph.run(vehicle_luke_query).data()

[{'class': 'airspeeder',
  'manufacturer': 'Incom corporation',
  'pilot': 'Wedge Antilles',
  'vehicle': 'Snowspeeder'},
 {'class': 'speeder',
  'manufacturer': 'Aratech Repulsor Company',
  'pilot': 'Leia Organa',
  'vehicle': 'Imperial Speeder Bike'}]